## Coursera Capstone assignment - Toronto neighbourhood Clustering & Segmenting - Part 2

This notebook is created as part of Corusera Capstone project. This notebook is created as part of capstone project  assignment - 'Clustering & Segmenting Toronto city data'. This notebook implements the following:

1. Extract the post codes & Neighbourhood data correspoding to various bouroughs in Toronto, ontario from a wikipedia page using python library - BeautifulSoup
    
2. Store the extracted data into a dataframe with columns as below:
    - Postal Code
    - Borough
    - Neighbourhood
    
3. **Update the above created dataframe with the Latitude & Longitudes details**

**Note**:
The suggested geocoder library is not found to be providing the latitude longitude values. Hence using the geospatial values to fetch teh requried latitude longitude values.

To add the lat long values to the toronto neighborhood dataframe, the following steps will be done in this notebook

1. Fetch the latlong data into a dataframe from the given web location
1. Sort the latlong dataframe based on post code, drop the post code entry
1. Sort the toronto neighbourhood dataframe with postcode
1. Add the latlong dataframe into the toronto neighbourhood dataframe.
 

### Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#import BeautifulSoup library for parsing wikipedia page
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


## This section repeats the neighbourhood data dataset creation from assignment part 1 ie scraping the wikipage & storing it in dataframe

In [2]:
wiki_page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc =  requests.get(wiki_page)
wikipage = BeautifulSoup(html_doc.text, 'html.parser') #read the extracted Wikipage using beautifulsoup
table = wikipage.find("table")
i=0
j=0
temp_dict=dict()
for row in table.findAll("tr"): 
    col = row.findAll("td")
    if len(col)>0:
        pcode = col[0].get_text().strip('\n')
        borough = col[1].get_text().strip('\n')
        neighbourhood = col[2].get_text().strip('\n')
        dict_key=pcode+'_'+borough
        
        if neighbourhood == 'Not assigned':
            neighbourhood = borough # to handle neighbouhoods that are 'Not assgined' & assign them to borough value in such cases
            
        if borough != 'Not assigned':  #skips all boroughs which are having value = 'Not assigned'
            #group the neighbourhoods belonging to same postal area in a comma separated csv
            if dict_key in temp_dict:
                temp_dict[dict_key]=temp_dict[dict_key]+','+neighbourhood
            else:
                temp_dict[dict_key]=neighbourhood
        else:
            j=j+1
        i=i+1
#priting howmany entries skipped        
text = 'Skipped {} boroughs out of {} entries as they are <Not assigned>\n'.format(j,i)
print(text)

#convert dictionary to a temporary dataframe
temp_df = pd.DataFrame(temp_dict.items(),columns=['key','Neighbourhood'])
#Split the key column to separate columns
temp_df[['Postal Code','Borough']] = temp_df['key'].str.split('_', n=1, expand=True)
# drop the key column
temp_df.drop('key',axis=1,inplace=True)

#Rearrange the columns and store the data to the base dataframe for toronto neighbourhoods
toronto_nhood_data = temp_df[['Postal Code','Borough','Neighbourhood']]
print("Toronto neighbourhoods dataframe created!!\n")
print('Dataframe shape:',toronto_nhood_data.shape,'\n')
toronto_nhood_data.head()

Skipped 77 boroughs out of 289 entries as they are <Not assigned>

Toronto neighbourhoods dataframe created!!

Dataframe shape: (103, 3) 



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Append the toronto neighbourhood dataframe with lat long values

### Fetch the geospatial data from the provide web location into a pandas dataframe

In [3]:
lat_long_data=pd.read_csv('http://cocl.us/Geospatial_data')
print("Lat-long data fetched\n")
print('Dataframe shape:',lat_long_data.shape,'\n')
print("Sorting the lat-long dataframe with PostCode...")
lat_long_data = lat_long_data.sort_values(by=['Postal Code'])
print("Dropping the  the lat-long dataframe with PostCode...")
lat_long_data.drop('Postal Code',axis=1,inplace=True)
lat_long_data.head()

Lat-long data fetched

Dataframe shape: (103, 3) 

Sorting the lat-long dataframe with PostCode...
Dropping the  the lat-long dataframe with PostCode...


,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


###  Sort the toronto neighbourhood data with postal code 

In [4]:
toronto_nhood_data=toronto_nhood_data.sort_values(by=['Postal Code'])
toronto_nhood_data.reset_index(drop=True,inplace=True)
print("Toronto neighbourhood dataframe sorted with PostCode!")
toronto_nhood_data.head()

Toronto neighbourhood dataframe sorted with PostCode!


,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Add the latlong dataframe into toronto neighbourhood dataframe to get the final dataframe

In [5]:
toronto_nhood_data [['Latitude','Longitude']] = lat_long_data
toronto_nhood_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Verify the final dataframe

In [6]:
print('Dataframe shape:',toronto_nhood_data.shape,'\n')

toronto_nhood_data[toronto_nhood_data['Postal Code']=='M9V']

Dataframe shape: (103, 5) 



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
